## Option 1: Auto-refresh cache for a set of Looks
First, some boring setup stuff. We import the API and create a connection client.

In [ ]:
import lookCache
import swagger_client as looker
client = lookCache.apiConnect('config.yml')

2017-05-02 19:12:54,553 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1109a1f98>: Failed to establish a new connection: [Errno 60] Operation timed out',)': /api/3.0/login?client_id=z4NM6BcrDZpCqrkcwdjg&client_secret=mnJxk59SKsZW5fdHnbDGpgVY


This next bit creates the looker objects we'll use to do all the neat stuff.

In [ ]:
look = looker.LookApi(client)
space = looker.SpaceApi(client)

Now we generate a list of all spaces. We can use a list comprehension to easily filter to what we want, in this case all public spaces. Note we have to cast the results to integers.

In [ ]:
allSpace = space.all_spaces()
targetSpaces = [int(i.id) for i in allSpace if i.id != 'lookml' and i.is_personal == False]

In [ ]:
print(targetSpaces)

Now we can loop through this list of spaces, extract all the looks contained therein...

In [ ]:
targetLooks = []
for s in targetSpaces:
    spaceLook = space.space_looks(s)
    spaceLookId = [i.id for i in spaceLook]
    targetLooks.append(spaceLookId)


...and flatten it into a single list. If this gets really large you may want to yield a generator instead of a list.

In [ ]:
flattened = [item for sublist in targetLooks for item in sublist]
print(flattened)

Now we could loop through this list of looks and run each one with cache set to False. In the interest of time we'll only do one...

In [ ]:
look.run_look(look_id = 173, result_format='json', cache = False)

## Option 2: Utilize dashboard pre-fetching
We can also utilize Looker's dashboard prefetch ability. If you have a heavily used dashboard with a set number of common combinations of filters this can be a solid option. But it does require some setup. First, we need to create a dashboard object.

In [ ]:
dash = looker.DashboardApi(client)

By default, dashboards are not setup for prefetching. We need to update our dashboard's configuration

In [ ]:
dash.update_dashboard(dashboard_id=9, body={"load_configuration":"prefetch_cache_run"})

Now we can create a couple of prefetches. After we run this we'll switch over to the dashboard to see it in action

In [ ]:
dash.create_dashboard_prefetch(dashboard_id=9, 
                               body={"ttl": 600,
                                     "dashboard_filters": [
                                        {
                                        "name": "Date",
                                        "value":"3 months ago for 3 months",
                                        "can": {}
                                        }
                                     ]
                                    }
                              )

dash.create_dashboard_prefetch(dashboard_id=9, 
                               body={"ttl": 600,
                                     "dashboard_filters": [
                                        {
                                        "name": "Date",
                                        "value":"5 months ago for 5 months",
                                        "can": {}
                                        }
                                     ]
                                    }
                              )

Let's check on the status of our '3 month' prefetch. Note the 'hit-count' metric... we'll come back to this in a moment.

In [ ]:
dash.dashboard_prefetch(dashboard_id = 9, 
                        dashboard_filters = '[{ "name": "Date", "value":"3 months ago for 3 months", "can": {} }]'
                       )

Hope that inspires some hacking. Have fun!